In [1]:
## Setting the PySpark environment variables

# import os

# os.environ['SPARK_HOME'] = "/opt/spark/spark-3.5.1-bin-hadoop3"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
from pyspark.sql import SparkSession

# from pyspark.sql.functions import *
from pyspark.sql.functions import desc

In [3]:
# Create a SparkSession object

spark = SparkSession.builder.appName("DataFrame-SQL-Demo").getOrCreate()

24/03/19 21:53:09 WARN Utils: Your hostname, dhiraj resolves to a loopback address: 127.0.1.1; using 192.168.10.66 instead (on interface wlo1)
24/03/19 21:53:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/19 21:53:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Simply view the .csv data using `basch command`

In [4]:
%%bash
head -10 ../data/csv/persons.csv

name,age,gender,salary
John Doe,30,Male,50000
Jane Smith,25,Female,45000
David Johnson,35,Male,60000
Emily Davis,28,Female,52000
Michael Wilson,40,Male,75000
Sarah Brown,32,Female,58000
Robert Lee,29,Male,51000
Lisa Garcia,27,Female,49000
James Martinez,38,Male,70000


### Load `.csv` file data into `Dataframe`

In [5]:
# Load the data into a DataFrame

data_file_path = "../data/csv/persons.csv"

df = spark.read.csv(data_file_path, header=True, inferSchema=True)

In [6]:
# Display schema of DataFrame
df.printSchema()

# Show the initial DataFrame
print("Initial DataFrame:")
df.show(10)

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

Initial DataFrame:
+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|        Jane Smith| 25|Female| 45000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
+------------------+---+------+------+
only showing top 10 rows



### Create the `Temporary View/Table` from dataframe

In [7]:
# Register the DataFrame as a Temporary Table

# Here, my_table is a name of the view
df.createOrReplaceTempView("my_table")

### Performing Spark `SQL like query`

In [8]:
# Spark SQL query to Select all rows where age is greater than 25

result = spark.sql("SELECT * FROM my_table WHERE age > 25")

result.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [9]:
# Compute the average salary by gender

avg_salary_by_gender = spark.sql("SELECT gender, AVG(salary) as avg_salary FROM my_table GROUP BY gender")

avg_salary_by_gender.show()

+------+----------+
|gender|avg_salary|
+------+----------+
|Female|   52300.0|
|  Male|   62100.0|
+------+----------+



### Creating and Managing Temporary views

In [10]:
# Create a temporary view/table from dataframe

df.createOrReplaceTempView("people")

In [11]:
# Query the temporary view

result = spark.sql("SELECT * FROM people WHERE age > 25")

result.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [12]:
# Check if a temporary view exists

view_exists = spark.catalog.tableExists("people")

view_exists

True

In [13]:
# Drop a temporary view

spark.catalog.dropTempView("people")

True

In [14]:
# Check if a temporary view exists

view_exists = spark.catalog.tableExists("people")

view_exists

False

In [15]:
# Stop the SparkSession

spark.stop()